In [452]:
import pandas
import numpy
from scipy import stats

## Hybrid sample

In [453]:
filepath = 'D:/BG/Data/Processing/4_data_hybrid.txt'
hybrid = pandas.read_csv(filepath, sep='\t')
print(hybrid.shape[0],'job postings in hybrid sample')
hybrid.columns

212821 job postings in hybrid sample


Index(['BGTJobId', 'JobDate', 'CleanTitle', 'CanonTitle', 'SOC', 'SOCName',
       'ONET', 'ONETName', 'Employer', 'Sector', 'SectorName', 'NAICS3',
       'NAICS4', 'NAICS5', 'NAICS6', 'City', 'County', 'State', 'Lat', 'Lon',
       'Edu', 'MaxEdu', 'Degree', 'MaxDegree', 'Exp', 'MaxExp', 'MinSalary',
       'MaxSalary', 'MinHrlySalary', 'MaxHrlySalary', 'PayFrequency',
       'SalaryType', 'JobHours', 'SOC2', 'SOC4', 'SOC5', 'SK',
       'Employer_clean', 'n_terms', 'SK_without', 'TECH', 'occupation',
       'firm1', 'firm2', 'firm', 'plant', 'plant_all', 'plant_5',
       'development', 'materials', 'design', 'inventory', 'tooling',
       'automation', 'production', 'maintenance', 'technical',
       'administrative', 'management', 'finance', 'business', 'data',
       'software', 'office', 'ml', 'cognitive', 'creativity', 'social',
       'character', 'customer', 'writing', 'nonroutine analytic',
       'nonroutine manual', 'routine cognitive', 'routine manual',
       'sequential

In [459]:
mydf = hybrid.copy()
mydf['Count of Terms']=[len(job.split(',')) for job in mydf.SK_without.tolist()]
annotation = ['nonroutine analytic','nonroutine manual','routine cognitive','routine manual','reciprocal','sequential']
for a in annotation:
  mydf[a] = (mydf[a]*mydf['Count of Terms'].sum())/(mydf['Count of Terms']*mydf[a].sum())

## Statistics for all occupations

In [460]:
df = mydf[['TECH'] + annotation]

t = []
for a in annotation:
  t.append([a,stats.ttest_ind(df.loc[df.TECH=='AM',a],df.loc[df.TECH=='TM',a],equal_var=False)[1]])
tdf = pandas.DataFrame(t,columns=['annotation','p-val'])
tdf.set_index('annotation',inplace=True) 

mymean = df.groupby(by=['TECH']).mean().T
mymean.index.name = 'annotation'
mymean['AM-TM'] = mymean['AM'] - mymean['TM']

mymean['N-AM'] = N = len(mydf.loc[mydf.TECH=='AM'])
mymean['N-TM'] = N = len(mydf.loc[mydf.TECH=='TM'])

table1 = pandas.merge(left=mymean,right=tdf,how='inner',left_on=['annotation'],right_on=['annotation'])
table1['statistics'] = 'Mean'
table1.set_index('statistics',append=True,inplace=True)

mysd = df.groupby(by=['TECH']).std(ddof=0).T
mysd.index.name = 'annotation'
mysd.reset_index(inplace=True)
mysd['statistics'] = 'SD'
mysd.set_index(['annotation','statistics'],inplace=True)
mysd['p-val'] = numpy.nan
table1 = pandas.concat([table1,mysd],axis=0).sort_index(kind='merge').reindex(annotation,axis=0,level='annotation')
table1.columns = pandas.MultiIndex.from_product([['All'],table1.columns])

postings = mydf.groupby(['TECH']).nunique()[['BGTJobId']].T
postings.rename_axis([None],axis=1,inplace=True)
postings.columns = pandas.MultiIndex.from_product([['All'],postings.columns])
postings.index = pandas.MultiIndex.from_product([['Number of job postings'],['']])
plants = mydf.groupby(['TECH']).nunique()[['plant']].T
plants.rename_axis([None],axis=1,inplace=True)
plants.columns = pandas.MultiIndex.from_product([['All'],plants.columns])
plants.index = pandas.MultiIndex.from_product([['Number of plants'],['']])
table1 = pandas.concat([table1,postings,plants],axis=0)
table1

All                              \
                                            AM     AM-TM    N-AM      N-TM   
annotation             statistics                                            
nonroutine analytic    Mean           1.323858  0.355728  4638.0  208183.0   
                       SD             1.219334       NaN     NaN       NaN   
nonroutine manual      Mean           0.780737 -0.610377  4638.0  208183.0   
                       SD             3.281382       NaN     NaN       NaN   
routine cognitive      Mean           0.667039 -0.407509  4638.0  208183.0   
                       SD             3.638618       NaN     NaN       NaN   
routine manual         Mean           1.283783  0.188751  4638.0  208183.0   
                       SD             1.939213       NaN     NaN       NaN   
reciprocal             Mean           1.024896  0.093623  4638.0  208183.0   
                       SD             3.481768       NaN     NaN       NaN   
sequential             Mean           0.709707 -0.303935  4638.0  208183.0   
                       SD             0.973196       NaN     NaN       NaN   
Number of job postings             4638.000000       NaN     NaN       NaN   
Number of plants                    319.000000       NaN     NaN       NaN   

                                                                
                                              TM         p-val  
annotation             statistics                               
nonroutine analytic    Mean             0.968130  9.561266e-83  
                       SD               1.225981           NaN  
nonroutine manual      Mean             1.391114  1.612789e-34  
                       SD               5.059892           NaN  
routine cognitive      Mean             1.074547  1.303517e-13  
                       SD               5.703446           NaN  
routine manual         Mean             1.095033  6.325611e-11  
                       SD               2.003587           NaN  
reciprocal             Mean             0.931273  7.034271e-02  
                       SD               3.559670           NaN  
sequential             Mean             1.013641  3.993299e-93  
                       SD               1.207701           NaN  
Number of job postings             208183.000000           NaN  
Number of plants                      319.000000           NaN

## Statistics by occupation

In [461]:
occupations = ['Manager','Engineer','Technician','Operator']
df = mydf[['TECH','occupation'] + annotation]

t = []
for a in annotation:
  for o in occupations:
    t.append([a,o,stats.ttest_ind(df.loc[(df.TECH=='AM') & (df.occupation==o),a],df.loc[(df.TECH=='TM') & (df.occupation==o),a],equal_var=False)[1]])
tdf = pandas.DataFrame(t,columns=['annotation','occupations','p-val'])
tdf.set_index(['annotation','occupations'],inplace=True)

mymean = df.groupby(by=['occupation','TECH']).mean().unstack(level=0).T
mymean.index.names = ['annotation','occupations']
mymean['AM-TM'] = mymean['AM'] - mymean['TM']

mymean.loc[pandas.IndexSlice[:,'Manager'],'N-AM'] = len(mydf.loc[(mydf.TECH=='AM')&(mydf.occupation=='Manager')])
mymean.loc[pandas.IndexSlice[:,'Engineer'],'N-AM'] = len(mydf.loc[(mydf.TECH=='AM')&(mydf.occupation=='Engineer')])
mymean.loc[pandas.IndexSlice[:,'Technician'],'N-AM'] = len(mydf.loc[(mydf.TECH=='AM')&(mydf.occupation=='Technician')])
mymean.loc[pandas.IndexSlice[:,'Operator'],'N-AM'] = len(mydf.loc[(mydf.TECH=='AM')&(mydf.occupation=='Operator')])
mymean.loc[pandas.IndexSlice[:,'Manager'],'N-TM'] = len(mydf.loc[(mydf.TECH=='TM')&(mydf.occupation=='Manager')])
mymean.loc[pandas.IndexSlice[:,'Engineer'],'N-TM'] = len(mydf.loc[(mydf.TECH=='TM')&(mydf.occupation=='Engineer')])
mymean.loc[pandas.IndexSlice[:,'Technician'],'N-TM'] = len(mydf.loc[(mydf.TECH=='TM')&(mydf.occupation=='Technician')])
mymean.loc[pandas.IndexSlice[:,'Operator'],'N-TM'] = len(mydf.loc[(mydf.TECH=='TM')&(mydf.occupation=='Operator')])

table2 = pandas.merge(left=mymean,right=tdf,how='inner',left_on=['annotation','occupations'],right_on=['annotation','occupations'])
table2['statistics'] = 'Mean'
table2.set_index('statistics',append=True,inplace=True)

mysd = df.groupby(by=['occupation','TECH']).std(ddof=0).unstack(level=0).T
mysd.index.names = ['annotation','occupations']
mysd['statistics'] = 'SD'
mysd.set_index('statistics',append=True,inplace=True)
mysd['p-val'] = numpy.nan

table2 = pandas.concat([table2,mysd],axis=0).sort_index(kind='merge').reindex(annotation,axis=0,level='annotation').unstack('occupations').swaplevel(0,1,axis=1).reindex(occupations,axis=1,level=0)
table2.rename_axis([None,None],axis=1,inplace=True)

postings = mydf.groupby(['occupation','TECH']).nunique()[['BGTJobId']].reindex(occupations, level='occupation').T
postings.rename_axis([None,None],axis=1,inplace=True)
postings.index = pandas.MultiIndex.from_product([['Number of job postings'],['']])
plants = mydf.groupby(['occupation','TECH']).nunique()[['plant']].reindex(occupations, level='occupation').T
plants.rename_axis([None,None],axis=1,inplace=True)
plants.index = pandas.MultiIndex.from_product([['Number of plants'],['']])
table2 = pandas.concat([table2,postings,plants],axis=0).reindex(occupations,axis=1,level=0)
table2

Manager                            \
                                           AM     AM-TM   N-AM     N-TM   
annotation             statistics                                         
nonroutine analytic    Mean          0.878229  0.171163  981.0  88819.0   
                       SD            1.024113       NaN    NaN      NaN   
nonroutine manual      Mean          0.080620 -0.357633  981.0  88819.0   
                       SD            0.885632       NaN    NaN      NaN   
routine cognitive      Mean          0.332608 -0.062424  981.0  88819.0   
                       SD            2.444517       NaN    NaN      NaN   
routine manual         Mean          1.294732  0.691326  981.0  88819.0   
                       SD            2.127794       NaN    NaN      NaN   
reciprocal             Mean          2.005284  0.564911  981.0  88819.0   
                       SD            5.036010       NaN    NaN      NaN   
sequential             Mean          1.045041 -0.296800  981.0  88819.0   
                       SD            1.135218       NaN    NaN      NaN   
Number of job postings             981.000000       NaN    NaN      NaN   
Number of plants                   166.000000       NaN    NaN      NaN   

                                                                  Engineer  \
                                             TM         p-val           AM   
annotation             statistics                                            
nonroutine analytic    Mean            0.707067  2.306313e-07     1.719291   
                       SD              0.918907           NaN     1.203884   
nonroutine manual      Mean            0.438253  9.580863e-32     0.468707   
                       SD              2.605293           NaN     2.060871   
routine cognitive      Mean            0.395032  4.281779e-01     0.634882   
                       SD              3.051619           NaN     3.208930   
routine manual         Mean            0.603406  4.220755e-23     0.897615   
                       SD              1.241125           NaN     1.310093   
reciprocal             Mean            1.440373  4.904876e-04     0.728477   
                       SD              4.320330           NaN     2.723635   
sequential             Mean            1.341841  1.232572e-15     0.612907   
                       SD              1.227781           NaN     0.877915   
Number of job postings             88819.000000           NaN  2704.000000   
Number of plants                     311.000000           NaN   274.000000   

                                                              ... Technician  \
                                      AM-TM    N-AM     N-TM  ...       N-AM   
annotation             statistics                             ...              
nonroutine analytic    Mean        0.164767  2704.0  78324.0  ...      457.0   
                       SD               NaN     NaN      NaN  ...        NaN   
nonroutine manual      Mean       -0.286497  2704.0  78324.0  ...      457.0   
                       SD               NaN     NaN      NaN  ...        NaN   
routine cognitive      Mean       -0.578329  2704.0  78324.0  ...      457.0   
                       SD               NaN     NaN      NaN  ...        NaN   
routine manual         Mean        0.033378  2704.0  78324.0  ...      457.0   
                       SD               NaN     NaN      NaN  ...        NaN   
reciprocal             Mean        0.114376  2704.0  78324.0  ...      457.0   
                       SD               NaN     NaN      NaN  ...        NaN   
sequential             Mean       -0.161358  2704.0  78324.0  ...      457.0   
                       SD               NaN     NaN      NaN  ...        NaN   
Number of job postings                  NaN     NaN      NaN  ...        NaN   
Number of plants                        NaN     NaN      NaN  ...        NaN   

                                                                        \


## Final table

In [462]:
table = pandas.merge(left=table1,right=table2,how='inner',left_on=['annotation','statistics'],right_on=['annotation','statistics'])
table[('Manager','Within')] = (table[('Manager','N-TM')]*table[('Manager','AM-TM')])/table[('All','N-TM')]
table[('Manager','Between')] = (table[('Manager','N-AM')]/table[('All','N-AM')]-table[('Manager','N-TM')]/table[('All','N-TM')])*table[('Manager','AM')]
table[('Engineer','Within')] = (table[('Engineer','N-TM')]*table['Engineer','AM-TM'])/table['All','N-TM']
table[('Engineer','Between')] = (table[('Engineer','N-AM')]/table[('All','N-AM')]-table[('Engineer','N-TM')]/table[('All','N-TM')])*table[('Engineer','AM')]
table[('Technician','Within')] = (table[('Technician','N-TM')]*table['Technician','AM-TM'])/table['All','N-TM']
table[('Technician','Between')] = (table[('Technician','N-AM')]/table[('All','N-AM')]-table[('Technician','N-TM')]/table[('All','N-TM')])*table[('Technician','AM')]
table[('Operator','Within')] = (table[('Operator','N-TM')]*table['Operator','AM-TM'])/table['All','N-TM']
table[('Operator','Between')] = (table[('Operator','N-AM')]/table[('All','N-AM')]-table[('Operator','N-TM')]/table[('All','N-TM')])*table[('Operator','AM')]
table[('All','Within-Check')] = table[('Manager','Within')]+table[('Engineer','Within')]+table[('Technician','Within')]+table[('Operator','Within')]
table[('All','Between')] = table[('Manager','Between')]+table[('Engineer','Between')]+table[('Technician','Between')]+table[('Operator','Between')]
table = numpy.trunc(1000*table)/1000
table[('All','Within')] = table[('All','AM')] - table[('All','TM')] - table['All','Between']
check = table[[('All','Within-Check'),('All','Within')]]
print(check) # because of decimal places; 'Within-Check' is calculated from the occupations, 'Within' is calculated from All to deal with decimal places

mylist = [('All','AM'),('All','TM'),('All','Between'),('All','Within'),('Manager','AM'),('Manager','TM'),('Engineer','AM'),('Engineer','TM'),('Technician','AM'),('Technician','TM'),('Operator','AM'),('Operator','TM')]
table[mylist] = table[mylist].applymap('{:.3f}'.format)

for i in ['All','Manager','Engineer','Technician','Operator']:
  table.loc[(table[(i,'p-val')]<0.1)&(table[(i,'p-val')]>=0.05),(i,'AM')] = table.loc[(table[(i,'p-val')]<0.1)&(table[(i,'p-val')]>=0.05),(i,'AM')]+'*'
  table.loc[(table[(i,'p-val')]<0.05)&(table[(i,'p-val')]>=0.01),(i,'AM')] = table.loc[(table[(i,'p-val')]<0.05)&(table[(i,'p-val')]>=0.01),(i,'AM')]+'**'
  table.loc[table[(i,'p-val')]<0.01,(i,'AM')] = table.loc[table[(i,'p-val')]<0.01,(i,'AM')]+'***'

table = table[mylist]
table.loc[pandas.IndexSlice[:,'SD'],:] = table.loc[pandas.IndexSlice[:,'SD'],:].apply(lambda x: '(' + x + ')')
table = table.mask(table == '(nan)',numpy.nan)
table = table.mask(table == 'nan',numpy.nan)
table = table.mask(table == '-0.000','0.000')
table.loc[pandas.IndexSlice['Number of job postings',:],:] = table.loc[pandas.IndexSlice['Number of job postings',:],:].replace(r'.000','',regex=True)
table.loc[pandas.IndexSlice['Number of plants',:],:] = table.loc[pandas.IndexSlice['Number of plants',:],:].replace(r'.000','',regex=True)

table

                                           All       
                                  Within-Check Within
annotation             statistics                    
nonroutine analytic    Mean              0.173  0.173
                       SD                  NaN    NaN
nonroutine manual      Mean             -0.702 -0.703
                       SD                  NaN    NaN
routine cognitive      Mean             -0.479 -0.479
                       SD                  NaN    NaN
routine manual         Mean              0.254  0.253
                       SD                  NaN    NaN
reciprocal             Mean              0.350  0.350
                       SD                  NaN    NaN
sequential             Mean             -0.207 -0.208
                       SD                  NaN    NaN
Number of job postings                     NaN    NaN
Number of plants                           NaN    NaN


All                           \
                                         AM       TM Between  Within   
annotation             statistics                                      
nonroutine analytic    Mean        1.323***    0.968   0.182   0.173   
                       SD           (1.219)  (1.225)     NaN     NaN   
nonroutine manual      Mean        0.780***    1.391   0.092  -0.703   
                       SD           (3.281)  (5.059)     NaN     NaN   
routine cognitive      Mean        0.667***    1.074   0.072  -0.479   
                       SD           (3.638)  (5.703)     NaN     NaN   
routine manual         Mean        1.283***    1.095  -0.065   0.253   
                       SD           (1.939)  (2.003)     NaN     NaN   
reciprocal             Mean          1.024*    0.931  -0.257   0.350   
                       SD           (3.481)  (3.559)     NaN     NaN   
sequential             Mean        0.709***    1.013  -0.096  -0.208   
                       SD           (0.973)  (1.207)     NaN     NaN   
Number of job postings                 4638   208183     NaN     NaN   
Number of plants                        319      319     NaN     NaN   

                                    Manager           Engineer           \
                                         AM       TM        AM       TM   
annotation             statistics                                         
nonroutine analytic    Mean        0.878***    0.707  1.719***    1.554   
                       SD           (1.024)  (0.918)   (1.203)  (1.427)   
nonroutine manual      Mean        0.080***    0.438  0.468***    0.755   
                       SD           (0.885)  (2.605)   (2.060)  (3.500)   
routine cognitive      Mean           0.332    0.395  0.634***    1.213   
                       SD           (2.444)  (3.051)   (3.208)  (5.783)   
routine manual         Mean        1.294***    0.603     0.897    0.864   
                       SD           (2.127)  (1.241)   (1.310)  (1.466)   
reciprocal             Mean        2.005***    1.440   0.728**    0.614   
                       SD           (5.036)  (4.320)   (2.723)  (3.002)   
sequential             Mean        1.045***    1.341  0.612***    0.774   
                       SD           (1.135)  (1.227)   (0.877)  (1.084)   
Number of job postings                  981    88819      2704    78324   
Number of plants                        166      311       274      309   

                                  Technician           Operator           
                                          AM       TM        AM       TM  
annotation             statistics                                         
nonroutine analytic    Mean            0.781    0.736  0.549***    0.299  
                       SD            (1.047)  (1.189)   (0.894)  (0.724)  
nonroutine manual      Mean         2.298***    4.052  2.468***    4.886  
                       SD            (5.845)  (7.437)   (6.166)  (9.293)  
routine cognitive      Mean         1.144***    1.913  1.063***    2.411  
                       SD            (4.945)  (8.195)   (5.719)  (9.084)  
routine manual         Mean         2.458***    2.042  2.285***    2.798  
                       SD            (2.705)  (2.522)   (2.650)  (3.346)  
reciprocal             Mean         1.056***    0.388    0.672*    0.451  
                       SD            (3.757)  (2.218)   (2.691)  (2.437)  
sequential             Mean         0.595***    0.717    0.679*    0.775  
                       SD            (0.824)  (1.177)   (1.091)  (1.233)  
Number of job postings                   457    10791       496    30249  
Number of plants                         104      273       131      300

In [463]:
table.to_excel('Appendix-Table2-terms.xlsx')